<a href="https://colab.research.google.com/github/csesoumita/FingerprintClassification/blob/master/DL_Assignment_2422493c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


                          
**Name:** Soumita Chel

**Student Id:** 2422493c

**Course:** Data Science


**Loading of DataSet**

**Google Drive Link For DataSet: ** https://drive.google.com/open?id=1l5xNP3mYp-9rLlWHbHorN77gg-Fd1R1W

In [0]:
#Loading The Data Set in path /content which is also the present working directory
from google.colab import files
files.upload()

Saving DataSet.rar to DataSet.rar


**Unzipping the DataSet **

In [0]:
#The uploaded data set is in .rar format. Below is the command to unrar it.
!unrar x DataSet.rar
#!unzip DataSet.zip


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from DataSet.rar

Creating    DataSet                                                   OK
Creating    DataSet/Training                                          OK
Creating    DataSet/Training/Female                                   OK
Extracting  DataSet/Training/Female/130__F_Right_thumb_finger.BMP          0%  OK 
Extracting  DataSet/Training/Female/135__F_Left_index_finger.BMP           0%  OK 
Extracting  DataSet/Training/Female/135__F_Left_little_finger.BMP          0%  OK 
Extracting  DataSet/Training/Female/135__F_Left_middle_finger.BMP          0%  OK 
Extracting  DataSet/Training/Female/135__F_Left_ring_finger.BMP            0%  OK 
Extracting  DataSet/Training/Female/135__F_Left_thumb_finger.BMP           0%  OK 
Extracting  DataSet/Training/Female/135__F_Right_index_finger.BMP          0%  OK 
Extracting  DataSet/Training/Female/135

**1.Analysis of the problem. **
Problem: The expected Deep Learning Model distinguishes between Male and Female fingerprints.
Dataset has been originally downloaded from https://arxiv.org/abs/1807.10609.

Then, dataset for this problem is created as per below statistics. 
(Training Data is approximately 70 percent and Validation Data is 30 percent of total data)

**Train Data**
********************************
**Male:**        686

**Female: **   796

**Validation Data**
********************************
**Male:**       268

**Female:**   497

**Test Data** is assumed to be the same as validation data with batchsize=1


In [0]:
#---------------Count of  Data ------------------------
! echo "Training Male Class: " 
!ls -lrt /content/DataSet/Training/Male | wc -l
! echo "Training Female Class: "
!ls -lrt /content/DataSet/Training/Female | wc -l
! echo "Test Male Class: " 
!ls -lrt /content/DataSet/Validation/Male | wc -l
! echo "Test Female Class: " 
!ls -lrt /content/DataSet/Validation/Female | wc -l

Training Male Class: 
686
Training Female Class: 
796
Test Male Class: 
268
Test Female Class: 
497


In [0]:
#Import of  the necessary libraries Start
import numpy as np
import matplotlib.pyplot  as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from keras.applications.resnet50 import preprocess_input, decode_predictions, ResNet50
from keras.applications.resnet50 import ResNet50
from google.colab import files
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import GlobalAveragePooling2D
#Import of the necessary libraries End 

Using TensorFlow backend.


**2.Visualisation and analysis of the data type**

The dataset contains images of fingerprints of both male and female.
They are grayscale images.
The size of the images are all same but they have been changed later with respect to the predefined model used.

**Class Distribution**
The model does binary classification. There are two classes i.e. Male and Female. The training data contains approximately same number of images to prevent class imbalance.

** Data Augmentation**
Preprocessing is done into the images present in the above dataset to prevent scenarios of overfitting and underfitting.

1) **Featurewise_center** - Used for standarization of pixels across the dataset. This is set to 'True' for this model.

2)** Vertical_flip -**  This is a type of random flips to improve performance.This is set to 'True' for this model.

3)** preprocessing_function** - This works on one image. It is implemented after the image is resized and augmented.




In [0]:
#Size of the image
imagesize=224

#Creation of DataGenerator Start 

#Data Augmentation of Training Data Start 
train_generator=ImageDataGenerator(featurewise_center=True, 
                                   vertical_flip=True,
                                   preprocessing_function=preprocess_input 
                                   )
train_gen=train_generator.flow_from_directory('DataSet/Training',target_size=(224,224),batch_size=10,class_mode='binary')
#Data Augmentation of Training Data Stop


#Validation Data Generator Creation
validation_generator=ImageDataGenerator(preprocessing_function=preprocess_input)
validation_gen=validation_generator.flow_from_directory(
 'DataSet/Validation',
  target_size=(imagesize,imagesize),
  batch_size=10,
  class_mode='binary'
)

Found 1478 images belonging to 2 classes.
Found 762 images belonging to 2 classes.


**3.Creation of multiple candidate network architectures**

Here, the  predefined model called ResNet50 is being used. 

Reason of using ResNet50:  It removes fully connected network at the end of the network.
Also implements skip connections.

**Layers**

Pooling Layers are added in between to reduce the number of parameters and computations steps in the network.The most common approach used in pooling is max pooling.Its main aim is to reduce the dimensions of images or layers , providing assumptions of the features of the intermediate layers.Here we have used two dimensional GlobalAveragePooling.It allows the images considered to be of any size. It takes the average of every incoming feature map and gives a standard tensor output. It  is to avoid any   requirement of parameter optimization in the intermediate layers. Another Deep Layer is added to train more number of nuerons.

**Activation Function **
It decides whether a nueron should be activated or not , based on the fact that if it contains the relevant set of information or not.
'relu' and 'softmax' are the two activation functions used here because of the following reasons:

1)** relu-** It does not activate all the nuerons at the same time.
If its input is negative, it will convert it to zero and the nueron would not be activated.

2) **softmax-** This is particularly useful for a classification problem, where it is trying to define the probability of each class prediction.




In [0]:
#####################Creating the model start ############################################
base_model =ResNet50(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x)
# and a logistic layer -- 
predictions = Dense(2, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

#####################Creating the model end ############################################


Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step


**Training:Optimization**

In [0]:
#Compilation of Model with Optimizer

from keras.optimizers import Nadam
model.compile(optimizer=Nadam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004), metrics=['accuracy'],loss='sparse_categorical_crossentropy')


**Training: Hyperparameter**

1) **Epoch:** An epoch is a single step where a nueral network is trained with the trained samples.
Here we have taken 10 such epochs.

2) **Steps_per_epoch** is number of iterations of batches before a single epoch is completed.

3)** Batch-size** is the number of training samples in each mini batch.

Here we have considered 3 steps_per_epoch for 10 epochs , as we could see increase in accuracy on subsequent trainings.
The accuracy was less about 20 percent when we considered steps_per_epoch to be 1.
For training and validation DataGenerator we have taken batchsize as 10.
For the test one, we have calculated on the basis of the data we have, to ultimately calculate confusion matrix for evaluation.

In [0]:
#Fitting the modlel for training
model.fit_generator(train_gen, 
                    steps_per_epoch=3,
                    epochs=10,
                    validation_data=validation_gen, 
                    validation_steps=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


3/3 [==============================] - 82s 27s/step - loss: 1.2971 - acc: 0.5667 - val_loss: 0.9816 - val_acc: 0.6300
Epoch 2/10
3/3 [==============================] - 69s 23s/step - loss: 0.8414 - acc: 0.6333 - val_loss: 1.0544 - val_acc: 0.6400
Epoch 3/10
3/3 [==============================] - 69s 23s/step - loss: 0.8769 - acc: 0.5333 - val_loss: 2.0634 - val_acc: 0.5400
Epoch 4/10
3/3 [==============================] - 68s 23s/step - loss: 0.8853 - acc: 0.7667 - val_loss: 1.6456 - val_acc: 0.5500
Epoch 5/10
3/3 [==============================] - 69s 23s/step - loss: 0.9521 - acc: 0.5667 - val_loss: 1.4148 - val_acc: 0.5800
Epoch 6/10
3/3 [==============================] - 69s 23s/step - loss: 1.2461 - acc: 0.4667 - val_loss: 0.8614 - val_acc: 0.5700
Epoch 7/10
3/3 [==============================] - 69s 23s/step - loss: 1.0389 - acc: 0.6000 - val_loss: 1.2505 - val_acc: 0.5200
Epoch 8/10
3/3 [==============================] - 66s 22s/step - loss: 1.3093 - acc: 0.5333 - val_loss: 1.52

**Evaluation of Trained Model on Test DataSet**

In [0]:
#Evaluation of Trained Model Start
score_training=model.evaluate_generator(validation_gen, 
                         steps=10, 
                         verbose=0)
print("Loss of trained data :",score_training[0])
print("Accuracy of trained data :",score_training[1])
#Evaluation of Trained Model End


Loss of trained data : 2.2789595425128937
Accuracy of trained data : 0.6100000083446503


 **5. Empirical evaluation of performance, and potentially visualisation and analysis of the trained network. This should make good use of graphs and tables of results, confusion matrices etc to represent the relative performance of the different models. Explain why you chose the metrics you use.**

Below classification metrices are taken into consideration for evaluation of performance of the model.

**I) Accuracy Report: **
It counts the number of correct predictions.

Correct number of predictions is an important factor in case of evaluation of metrics, so this is taken into consieration.

**II) Classification Report**
It is a text report showing the major classification metrices like precision, recall and f1-score of all the classes.

a)** Precision**  is the number of images predicted to be in a particular class, had correct prediction.

b) **Recall** is the number of the  images actually belonging to a particular class was predicted to be in the same class.

c)** F1-score** is the harmonic mean of the precision and recall for a particular class.

d) **Support **is the number of true responses that lies on the particular class.

It gives an overview of the major classification metrices, so here this has been considered.

**III) Confusion Report**
It is one of the important classificiation metrices which gives the types of errors made by the model.  
The above metrices hide essential details required for better understanding of the performance. Hence this metric is taken into consideration.

IV) **Cohen’s kappa** is a statistical metric which is useful to solve the problem of multiclass and class imbalance in deep learning.
It's value is always less than equal to 1. It gives a better performance overview in case where we have a misleading accuracy score.




In [0]:
import numpy
#Size of the image
imagesize=224
#Creating Test Data from Validation Data with batchsize=1
test_generator=ImageDataGenerator()
test_gen=test_generator.flow_from_directory(
'DataSet/Validation/',
  target_size=(imagesize,imagesize),
  batch_size=1,
  shuffle=False
)    

#Prediction with Test Data Start
validation_generator_steps_per_epoch=numpy.math.ceil(test_gen.samples / test_gen.batch_size)
print(validation_generator_steps_per_epoch)

predicted_test=model.predict_generator(test_gen,validation_generator_steps_per_epoch)
predicted_test_con=np.argmax(predicted_test, axis=1)
true_class=test_gen.classes
#Prediction with Test Data End
#Creation of Accuracy Score
from sklearn.metrics import accuracy_score
print('-------------Accuracy Score----------------------')
print ( accuracy_score(true_class, predicted_test_con))
#Creation of Classification Report
#class_labels = list(test_generator.class_indices.keys()) 
from sklearn.metrics import classification_report
print('-------------Classiification Report----------------')
print (classification_report(true_class, predicted_test_con))
#Creation of Confusion Matrix
from sklearn.metrics import confusion_matrix
print('-------------Confusion Matrix-------------------')
print(confusion_matrix(true_class, predicted_test_con))
#Creation of  Cohen’s kappa Metric
print('-------------Cohen’s kappa Metric-------------------')
from sklearn.metrics import cohen_kappa_score
print(cohen_kappa_score(true_class, predicted_test_con))
#Jaccard similarity coefficient score
from sklearn.metrics import jaccard_similarity_score
print('-------------Jaccard similarity coefficient score-------------------')
print(jaccard_similarity_score(true_class, predicted_test_con))


Found 762 images belonging to 2 classes.
762
-------------Accuracy Score----------------------
0.6706036745406824
-------------Classiification Report----------------
              precision    recall  f1-score   support

           0       0.68      0.93      0.79       496
           1       0.59      0.18      0.28       266

   micro avg       0.67      0.67      0.67       762
   macro avg       0.64      0.56      0.53       762
weighted avg       0.65      0.67      0.61       762

-------------Confusion Matrix-------------------
[[463  33]
 [218  48]]
-------------Cohen’s kappa Metric-------------------
0.1358202076612357
-------------Jaccard similarity coefficient score-------------------
0.6706036745406824


**Report on the performance**

According to the results obtained from the above metrices we can see,

Confusion Matrix
********************************
Total: 762

True Negative(TN)=463

False Positive(FP)=33

False Negative(FN)=218

True Positive(TP)=48 



Hence we can calculate the below:-

1) True Positive Rate-=TP/(FN+TP)=48/266=0.18

2) False Positive Rate=FP/(TN+FP)=33/496=0.066

3) True Negative Rate=TN/(TN+FP)=463/496=0.93

4) Precision=TP/(FP+TP)=48/81=0.59

5) Prevalance=(FN+TP)/Total=266/762=0.34


**Suitability of the Final Network**

The model has an trained model accuracy of 61 percent(approx) and test model accuracy of 67 (approx).
The model deals with grayscale images of fingerprint, so this can be considered as a good trained model.
As future work, we can train data with more data than now, and check its efficiency.

**References**

1) http://thedatascientist.com/performance-measures-cohens-kappa-statistic/

2) https://datascience.stackexchange.com/questions/29719/how-to-set-batch-size-steps-per-epoch-and-validation-steps

3) https://keras.io

4) https://scikit-learn.org/stable/modules/model_evaluation.html

5)https://towardsdatascience.com/deep-learning-performance-cheat-sheet-21374b9c4f45

6) https://machinelearningmastery.com/image-augmentation-deep-learning-keras/

7) https://towardsdatascience.com/an-overview-of-image-classification-networks-3fb4ff6fa61b
